# Simulated Annealing for Mars Lander Episode-1 

### Import modules

In [1]:
import numpy as np
import re, math, random, pdb, json
import peforth
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.26
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.26 (<class 'str'>)


# Episode-1 

In [2]:
from typing import List, Optional, Callable, Tuple, Dict, Union  # OneNote2021 #_python_ #_typing_
GENE = List[int]  # (rotate,power)
GENOME = List[GENE]  # Genome 是一個 list 裡面都是 int
STATE = Dict[str,Union[int,float]]  # x,y,hspeed,power,.. etc 

gravity = -3.711        # constant on Mars
g = gravity             # 到了二維以上，重力需轉成向量
fuel0 = 500             # episode-1 


In [3]:
class State(STATE):

    def __init__(self,step=0,x=0,y=0,hspeed=0,
            vspeed=0,fuel=0,angle=0,power=0,force=0):
        self.step   = step
        self.x      = x
        self.y      = y
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.fuel   = fuel
        self.angle  = angle
        self.power  = power
        self.force  = force

    def next(self, angle:int, power:int):  
        # input angle,power 都是 offset 分別屬 -15~15 以及 -1,0,+1 
        assert power in [-1, 0, 1], 'expected input power is an offset [-1, 0, +1], given {}'.format(power)
        assert angle in range(-15,16), 'input angle range [-15:16], given {}'.format(angle)
        self.step   += 1
        self.angle  += angle
        self.angle   = -90 if self.angle < -90 else self.angle
        self.angle   =  90 if self.angle > 90 else self.angle
        self.power  += power
        self.power   = 0 if self.power < 0 else self.power
        self.power   = 4 if self.power > 4 else self.power
        self.fuel   -= self.power
        self.x       = self.x       # episode-1 不變
        self.hspeed  = self.hspeed  # episode-1 不變
        self.force   = g + self.power
        self.y       = self.y + self.vspeed + (1/2)*self.force
        self.vspeed += self.force
        return self  # 傳回值只是方便測試，可以這樣寫： %f state :> next(0,3.711) -->

    def __str__(self):  
        return str(self.__dict__)

state0 = State(0,0,2500,0,0,fuel0,0,0,0)  # episode-1 的起點 state 

# 測試兼範例
# 模擬自由落體
state = State(**state0.__dict__)
%f 36 [for] state :> next(0,0) [next] 
%f ( Free fall 墜毀 at step 36 ) state --> cr 
%f state constant 自由落體 // ( -- state ) {'step': 36, 'x': 0, 'y': 95.272, 'hspeed': 0, 'vspeed': -133.596, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)

# 模擬安全降落
state = State(**state0.__dict__)
%f 9 [for] state :> next(0,0) [next] 
%f 91 9 - [for] state :> next(0,1) [next] 
%f ( 9 步以後狂噴到 91 步安全著陸 ) state --> cr 
%f state constant 安全降落 // ( -- state ) {'step': 91, 'x': 0, 'y': 97.605, 'hspeed': 0, 'vspeed': -15.701, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.289} (<class '__main__.State'>)

# 模擬飛向太空
state = State(**state0.__dict__)
%f 83 [for] state :> next(0,1) [next] 
%f ( 83 步被判逸失 ) state -->
%f state constant 飛向太空 // ( -- state ) {'step': 83, 'x': 0, 'y': 3004.460, 'hspeed': 0, 'vspeed': 17.987, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.289} (<class '__main__.State'>)

( Free fall 墜毀 at step 36 ) state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)

( 9 步以後狂噴到 91 步安全著陸 ) state --> {'step': 91, 'x': 0, 'y': 97.6045000000063, 'hspeed': 0, 'vspeed': -15.700999999999876, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)

( 83 步被判逸失 ) state --> {'step': 83, 'x': 0, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


In [4]:
# 改寫 for Mars Lander 乍看以為要頭痛一會兒，其實 Mars Lander 的 genome 很簡單，就是一串 (int,int) 
# pairs 而已。前面基礎討論有提到，以 SA 而言就是 mutate 某一個 gene 也就是 command pair 而已。但是考慮 Mars
# Lander 的限制條件，以 epidode-1 而言，command 的改變要做的決定是 -1 0 +1 其中之一才對，而非 0~4. 

# 長度的問題，
# 我想的是需要另一個 function 不夠時直接 copy 最後一個 gene 頂替即可。
# 會發現不夠的是 call fitness function 的人，那就是由它處理。

def get_random_neighbour(genome: GENOME) -> None :
    neighbour = genome[:]  # Deep copy 利用 [:] slice copy, 若用 list(state) 不行！

    idx = random.randint(0,len(genome)-1) # 選 1 個 gene 
    cmd = random.randint(-1, 1)  # -1, 0, +1 當中選 1 個 thrust command (episode-1)

    neighbour[idx][1] = cmd
    return neighbour

In [5]:
# ○ 成功著陸的，油料剩越多的越好。
# ○ 一個油料耗盡一個墜毀哪個好？墜毀時的速度減到+-40 以內需要多少油料？
# ○ 油料耗盡時的高度距離與速度以自由落體觸地，然後把速度調整到+-40 以內需要多少油料？
# ○ 根據我之前對 y=100 (flat ground 的海拔高度) 的研究 該 state 來自 log 的最後一 state. 
#   當 Simulating 時，該 state 就是 y 首次跨越 flat ground 高度觸地著陸之時。
# ○ cost 必大於 0。 Score 用油料剩餘量計算，由 fuel0 扣下來可到負值。換算成 cost 就是 
#      cost = fuel0 - score
#   核算，當 score 為 fuel0 時 cost 為 0 無誤； 當 score 為 -fuel0 時 cost 為 2fuel0
#   甚至更高，也無誤。 直接用這個數字，配合 SA 的 Temperature 設定要用 visualized 
#   graphs 來訂定 即可。

def cost_of_state(state: STATE) -> int:
    # state = State(**state_all_power.__dict__) 
    score = None  # 有可能是負的，先給個看得出異常的初值。

    # 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
    if state.y <= 100:
        # 觸地了，進一步查看速度是否合格
        if state.vspeed >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
            # 安全降落
            score = state.fuel
        else:  # 墜毀
            # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
            while True:
                state.next(0,1)  # 有個調整的過程，不是直接填 4 ！
                if state.vspeed >= -40:
                    score = state.fuel
                    break
    else:  # 還在空中，飛走了或油料耗盡。
        # 先模擬 free fall 觸地。
        while True: 
            state.next(0,-1)  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
            if state.y <= 100: break
        # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
        while True:
            if state.vspeed >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
                # 達標了
                score = state.fuel
                break
            else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
                state.next(0,1)
    cost = fuel0 - score
    return cost 

# 自由落體 cost_of_state :> (pop()) --> 1374 (<class 'int'>)
# 飛向太空 cost_of_state :> (pop()) --> 1938 (<class 'int'>)
# 安全降落 cost_of_state :> (pop()) --> 322 (<class 'int'>)


In [6]:
# 模擬試跑 genome. Genome 可能太長或不夠，傳回訂正好的 genome 以及最後 state  
def simulate_of_genome(genome: GENOME) -> Tuple[GENOME,STATE]:
    state = State(**state0.__dict__)  # init state for simulation
    # 照 genome 跑到觸地、飛走、或油料耗盡時為止， genome 可能含有無效的長度要 cut 掉以免浪費時間嘗試。
    for i in range(len(genome)): 
        state.next(*genome[i])  # run 1 step 
        if state.y <= 100 or state.y > 3000 or state.fuel <= 0:  # 到底了 
            genome = genome[:i+1]  # 太長的尾巴切掉
            break
        
    # 看看 genome 是否太短需要補綴 genes 讓 lander 觸地。
    while state.y >= 100 and state.y < 3000 and state.fuel >= 0:
        genome.append(genome[-1][:])  # 注意！若不用 [:] slice copy 則 append 進去的都是同一個。
        state.next(*genome[-1])  # 簡單照抄最後一 step
        
    return genome,state

# 很妙，這產生「自由落體」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,0]])
%f cr ( 「自由落體」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) -->
    
# 而這產生「全速飛走」的 genome 與最後 state 
genome, last_state = simulate_of_genome([[0,1]])
%f cr ( 「全速飛走」 ) genome -->
%f last_state -->
%f last_state cost_of_state :> (pop()) --> cr


cr ( 「自由落體」 ) genome --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
last_state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)
last_state cost_of_state :> (pop()) --> 1374 (<class 'int'>)

cr ( 「全速飛走」 ) genome --> [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 

In [233]:
# fitness function 
# https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)
def sa(initial: GENOME) -> Tuple[GENOME,int,int] :
    current = initial
    current, current_last_state = simulate_of_genome(current)
    current_cost = cost_of_state(current_last_state)
    
    temp = 1.0  # (1/50)*(2*fuel0)  # <---- [ ] Hyperparameter, 來自 Einstein's problem 的心得，取總分的 1/15 
    num_iterations = 0

    for i in range(1000):  # 若想安全降落為止，可能不如有限次數換個 epoch 試試運氣
        neighbour = get_random_neighbour(current)
        neighbour, neighbour_last_state = simulate_of_genome(neighbour)
        neighbour_cost = cost_of_state(neighbour_last_state)
        
        cost_delta = neighbour_cost - current_cost

        if cost_delta < 0 or random.random() < math.exp(-cost_delta/temp):
            current, current_cost = neighbour, neighbour_cost
            print(current_cost, cost_delta, temp, math.exp(-cost_delta/temp))

        num_iterations += 1
        if num_iterations % 10 == 0 and temp > 0.10:  # <---- [ ] Hyperparameter
            temp -= 0.05  # <---- [ ] Hyperparameter

        if neighbour_last_state.vspeed <= -40 and neighbour_last_state.y <= 100:
            print('B i n g o !')
            break;
            
    return current, num_iterations, current_cost

In [5]:
def epoch(state=None):
    nationalities = [ 'dane',      'brit',   'swede',       'norwegian', 'german'    ]
    colours       = [ 'yellow',    'red',    'white',       'green',     'blue'      ]
    animals       = [ 'horse',     'cat',    'bird',        'fish',      'dog'       ]
    beverages     = [ 'water',     'tea',    'milk',        'coffee',    'root beer' ]
    cigars        = [ 'pall mall', 'prince', 'blue master', 'dunhill',   'blends'    ]

    random.shuffle(nationalities)
    random.shuffle(colours      )
    random.shuffle(animals      )
    random.shuffle(beverages    )
    random.shuffle(cigars       )
    
    attributes = [colours, nationalities, beverages, cigars, animals]

    NUM_HOUSES = 5
    initial = []
    s = '['
    
    if state: 
        initial = state; 
    else:
        for i in range(NUM_HOUSES):
            initial.append([attr[i] for attr in attributes])
    
    
    # random.seed(100)

    solution, iterations = sa(initial)

    for house in solution:
        s += str(house) + ',';
    s = s[:-1] + ']';
    return s,iterations

In [6]:
if __name__ == "__main__":
    ss = '';
    for i in range(10000):
        s, iter = epoch()
        ss += "{} Number of iterations:{}\n".format(s, iter)
        print('.', end="")
    %f ss char results.txt writeTextFile 
    %f .( Done !!! ) cr

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

http://localhost:8888/lab/workspaces/auto-m/tree/OneDrive/%E6%96%87%E4%BB%B6/Jupyter%20Notebooks/I%20study%20Pandas.ipynb

In [7]:
import pandas as pd

## 研究 pandas read data 的眾多方法

我看其中就是 pd.read_clipboard() 好用。先把資料抓進 clipboard 然後執行之即得。
研究 iterations 數的分布的目的是想藉以調整 Annealing Algorithm 的 hyper-parameter 

In [8]:
data = pd.read_clipboard()
data

,Data
0,8910
1,9532
2,6423
3,6776
4,22155
...,...
995,13802
996,6385
997,7525
998,8103


In [41]:
%f data :> describe() ==>
%f data :> Data.median() --> 

data :> describe() ==>
                 Data
count    1000.000000
mean    13933.352000
std     13448.507494
min      1242.000000
25%      6925.250000
50%      8452.000000
75%     15068.000000
max    113487.000000 (<class 'pandas.core.frame.DataFrame'>)
data :> Data.median() --> 8452.0 (<class 'numpy.float64'>)


In [45]:
data = pd.read_clipboard()
data

,Iterations
0,4851
1,6097
2,20168
3,10302
4,27273
...,...
9995,6986
9996,9056
9997,18983
9998,7096


In [47]:
%f data :> describe() ==>
%f data :> Iterations.median() --> 

data :> describe() ==>
          Iterations
count   10000.00000
mean    13178.39830
std     12065.95423
min        45.00000
25%      6995.75000
50%      8426.00000
75%     14400.25000
max    152908.00000 (<class 'pandas.core.frame.DataFrame'>)
data :> Iterations.median() --> 8426.0 (<class 'numpy.float64'>)


In [24]:
genome

[[0, -1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, -1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, -1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1]]

In [16]:
fuel0 = 500
genome =[[0,0],[0,0],[0,0]]

In [ ]:
%pdb on
genome,count,cost = sa(genome)
print();print(cost, count, genome)


Automatic pdb calling has been turned ON
1144 -193 1.0 6.589235162723882e+83
1129 -15 1.0 3269017.3724721107
1129 0 1.0 1.0
1103 -26 1.0 195729609428.83878
1081 -22 1.0 3584912846.131592
1081 0 0.95 1.0
669 -412 0.95 2.2215675347185444e+188
669 0 0.95 1.0
669 0 0.95 1.0
654 -15 0.95 7199151.171653874
654 0 0.7999999999999998 1.0
654 0 0.7999999999999998 1.0
643 -11 0.09999999999999969 5.92097202766669e+47
643 0 0.09999999999999969 1.0
643 0 0.09999999999999969 1.0
628 -15 0.09999999999999969 1.3937095806670532e+65
628 0 0.09999999999999969 1.0
628 0 0.09999999999999969 1.0
628 0 0.09999999999999969 1.0
628 0 0.09999999999999969 1.0


OverflowError: math range error

> <ipython-input-233-67fabe07e0a3>(20)sa()
     18         if cost_delta < 0 or random.random() < math.exp(-cost_delta/temp):
     19             current, current_cost = neighbour, neighbour_cost
---> 20             print(current_cost, cost_delta, temp, math.exp(-cost_delta/temp))
     21 
     22         num_iterations += 1



ipdb>  math.exp(-cost_delta/temp))


*** SyntaxError: invalid syntax


...

1.0142320547350045e+304


ipdb>  math.exp(800)


*** OverflowError: math range error


ipdb>  math.exp(730)


*** OverflowError: math range error


In [17]:
genome

[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, -1],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 1],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 1],
 [0, 0],
 [0, 0],
 [0, 1],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0]]

In [13]:
test(*[11,22])

11 22


In [14]:
simulate_of_genome([[0,0],[0.0],[0.0]])

TypeError: next() missing 1 required positional argument: 'power'

In [15]:
genome

[[0, 0], [0.0], [0.0]]